# Predição de Janelas de Lançamento Espacial
Este notebook contém a análise de dados meteorológicos (ERA5) e dados históricos de lançamentos espaciais.
O objetivo é demonstrar o pipeline de predição de viabilidade de janelas de lançamento.

In [ ]:
# --- Importações ---
import pandas as pd
import xarray as xr
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Carregar dados de lançamentos

In [ ]:
launches = pd.read_csv('../launches_geo.csv')
launches['launch_time_utc'] = pd.to_datetime(launches['launch_time_utc'])
launches.head()

## 2. Carregar dados ERA5 (NetCDF)
Exemplo com o arquivo de teste gerado pelo script.

In [ ]:
era5 = xr.open_dataset('../era5_test_launch.nc')
era5

## 3. Pré-processamento / Feature Engineering
Criando algumas features simples: média de vento, temperatura, pressão, precipitação.

In [ ]:
# Ajuste dos nomes de variáveis conforme seu NetCDF
features = pd.DataFrame({
    'wind_u_mean': era5['u10'].mean(dim='time').values.flatten(),
    'wind_v_mean': era5['v10'].mean(dim='time').values.flatten(),
    'temp2m_mean': era5['t2m'].mean(dim='time').values.flatten(),
    'precipitation_total': era5['tp'].sum(dim='time').values.flatten(),
    'surface_pressure_mean': era5['sp'].mean(dim='time').values.flatten()
})

features.head()

## 4. Criar variável alvo (dummy)
1 = lançamento bem-sucedido, 0 = falha (simulação para demo).

In [ ]:
y = (launches['Status Mission'] == 'Success').astype(int)
X = features.iloc[:len(y), :]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 5. Treinar modelo Gradient Boosting

In [ ]:
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

## 6. Matriz de Confusão

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

## 7. Próximos passos
- Testar diferentes janelas de tempo e features
- Avaliar modelos XGBoost e Regressão Logística
- Interpretar via SHAP para explicar decisões do modelo